# 4.4.1 OpenCascade Bottle-tutorial

Please consult the OCCT - documentation for more explanations:

https://dev.opencascade.org/doc/overview/html/occt__tutorial.html


In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo

In [ ]:
myHeight = 70
myWidth = 50
myThickness = 30

In [ ]:
pnt1 = Pnt(-myWidth / 2., 0, 0);
pnt2 = Pnt(-myWidth / 2., -myThickness / 4., 0);
pnt3 = Pnt(0, -myThickness / 2., 0);
pnt4 = Pnt(myWidth / 2., -myThickness / 4., 0);
pnt5 = Pnt(myWidth / 2., 0, 0);

In [ ]:
seg1 = Segment(pnt1, pnt2)
arc = ArcOfCircle(pnt2, pnt3, pnt4)
seg2 = Segment(pnt4, pnt5)

In [ ]:
wire = Wire ([seg1, arc, seg2])
mirrored_wire = wire.Mirror(Axis((0,0,0), X))
w = Wire([wire, mirrored_wire])

In [ ]:
f = Face (w)
body = f.Extrude( myHeight*Z )

In [ ]:
body = body.MakeFillet (body.edges, myThickness / 12.)
DrawGeo (body);

In [ ]:
from ngsolve import Mesh
from ngsolve.webgui import Draw

In [ ]:
geo = OCCGeometry(body)
mesh = Mesh(geo.GenerateMesh(maxh=5))

In [ ]:
Draw (mesh);

## Adding the neck:

In [ ]:
neckax = Axes(body.faces.Max(Z).center, Z)

In [ ]:
myNeckRadius = myThickness / 4.
myNeckHeight = myHeight / 10
neck = Cylinder(neckax, myNeckRadius, myNeckHeight);

In [ ]:
body = body+neck
DrawGeo (body);

In [ ]:
mesh = Mesh(OCCGeometry(body).GenerateMesh(maxh=5))
Draw (mesh);

find face with maximal z-coordinate:

In [ ]:
fmax = body.faces.Max(Z)
thickbody = body.MakeThickSolid([fmax], -myThickness / 50, 1.e-3)
DrawGeo (thickbody);

In [ ]:
mesh = Mesh(OCCGeometry(thickbody).GenerateMesh(maxh=3))
Draw (mesh);

## Defining the threading

In [ ]:
cyl1 = Cylinder(neckax, myNeckRadius * 0.99, 1).faces[0]
cyl2 = Cylinder(neckax, myNeckRadius * 1.05, 1).faces[0]

In [ ]:
import math
aPnt = Pnt(2.*math.pi, myNeckHeight / 2.)
aDir = Dir( 2.*math.pi, myNeckHeight / 4. )
anAx2d = gp_Ax2d(aPnt, aDir)

In [ ]:
aMajor = 2. * math.pi
aMinor = myNeckHeight / 10
arc1 = Ellipse(anAx2d, aMajor, aMinor).Trim(0, math.pi)
arc2 = Ellipse(anAx2d, aMajor, aMinor/4).Trim(0, math.pi)

In [ ]:
seg = Segment(arc1.start, arc1.end)

In [ ]:
wire1 = Wire( [Edge(arc1, cyl1), Edge(seg, cyl1)] )
wire2 = Wire( [Edge(arc2, cyl2), Edge(seg, cyl2)] )

In [ ]:
threading = ThruSections ([wire1, wire2])

In [ ]:
res = thickbody+threading
DrawGeo (res);

In [ ]:
# mesh = Mesh(OCCGeometry(res).GenerateMesh(maxh=3))
# Draw (mesh);